<a href="https://colab.research.google.com/github/andirenata/Projek_UAS_DataMining/blob/main/Projek_UAS_Data_Mining_Kelompok_Korupsi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Nama Anggota Kelompok**

1. Alan Diki Saputra - 1806208283
2. Andi Renata Fadhillah - 1806207841
3. Kinanti Jati Kinasih - 1806144456
4. Niken Nabila - 1806207923
5. Wulan Hapsari Bhagyawanti - 1806208131

**Tugas** : mengambil data secara langsung dari media sosial twitter dengan topik "korupsi". Dilakukan proses Data Mining untuk mengekstrak insight/informasi dari data tersebut.

# **Import Package**

In [ ]:
import warnings; warnings.simplefilter('ignore')
import nltk

try:
    import google.colab
    IN_COLAB = True
    !wget https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/taudataDDGsna.py
    !wget https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/taudata_sna.py
    !mkdir data   
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/slang.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/stopwords_id.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/stopwords_en.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/kata_dasar.txt
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/wn-ind-def.tab
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/wn-msa-all.tab
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/ind_SA.csv
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/all_indo_man_tag_corpus_model.crf.tagger

    !pip install --upgrade spacy python-crfsuite unidecode textblob sastrawi sklearn-pycrfsuite
    !pip install --upgrade unidecode twython tweepy beautifulsoup4 json_lines lxml pyLDAvis
    !python -m spacy download en
    !python -m spacy download xx
    !python -m spacy download en_core_web_sm
    nltk.download('popular')
except:
    IN_COLAB = False
    print("Running the code locally, please make sure all the python module versions agree with colab environment and all data/assets downloaded")

In [ ]:
import taudataDDGsna as taudata
import tweepy, json, urllib.request, requests
from urllib.request import Request, urlopen
from twython import TwythonStreamer
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import graphviz
import time, numpy as np, matplotlib.pyplot as plt, pandas as pd, networkx as nx
import random, taudata_sna as tau
import pyLDAvis, pyLDAvis.sklearn; pyLDAvis.enable_notebook()
from graphviz import Digraph
import re

seed = 123
random.seed(seed)
np.random.seed(seed)
"Done"

# **Data**

In [ ]:
# Download data kumpulan tweets

!wget https://www.dropbox.com/s/tgt5ihcmnyeod5d/Tweets_2021_korupsi_kpk_koruptor.json?dl=0

In [4]:
def loadTweets(file='Tweets.json'):
    f=open(file,encoding='utf-8', errors ='ignore', mode='r')
    T=f.readlines();f.close()
    for i,t in enumerate(T):
        T[i] = json.loads(t.strip())
    return T

In [5]:
fileName = 'Tweets_2021_korupsi_kpk_koruptor.json?dl=0'
T2 = loadTweets(file=fileName)
print('tweet pertama oleh "{}" : "{}"'.format(T2[0]['user']['screen_name'],T2[0]['full_text']))

tweet pertama oleh "adhabangkit" : "RT @Anna82662230: Hari Anti Korupsi, Mahasiswa Kendari Las Pintu Pagar Kejati Sultra. 
Klo g mau nemuin pendemo sekalian gausah keluar... M…"


In [6]:
# Data tweets nya saja

D = [t['full_text'] for t in T2]
D[:5] # 5 tweet pertama

['RT @Anna82662230: Hari Anti Korupsi, Mahasiswa Kendari Las Pintu Pagar Kejati Sultra. \nKlo g mau nemuin pendemo sekalian gausah keluar... M…',
 'RT @chaterine_8: Tidak ada yg bisa menggaransi setiap langkah bisnis. \n\nBUMN hrs berani melakukan intervensi pasar dgn resiko bisnis krn in…',
 '@nofaria_cen @hemaviton99 @amandagempita @Rentonized @KPK_RI @KejaksaanRI @DivHumas_Polri @DPR_RI Thn 2018 gubernur nya si ahok? Atau siapa?',
 'Kejaksaan Agung harus berani menindak tegas korupsi yang dilakukan oleh Benny Tjokro dan Heru Hidayat dalam kasus Jiwasraya Skandalhukumjiwasraya',
 'Gegara Koruptor2 kakap dicokok hartanya pd disitain sm Kejaksaan Agung waktu KPK kmrn mandul cm ngerjain proyek recehan Apalagi kasus Jiwasraya amp Asabri pasti tersangkut nama org2 besar di pemerintahan sebelumnya']

# **Preprocessing**

In [9]:
# Menghilangkan username

remove_user = [re.sub(r'@\w+', '', x) for x in D]

In [10]:
# Mengatasi slang dan menghilangkan stopwords

slangFixId = taudata.loadCorpus(file = 'data/slang.txt', sep=':')
stopId, _ = taudata.LoadStopWords(lang='id')
stopId.add("rt")

list(stopId)[:5]

['kiranya', 'kenapa', 'entah', 'biasa', 'sebut']

In [11]:
cleanD = []
for t in tqdm(remove_user):
    doc = taudata.cleanText(t, fix=slangFixId, lan='id', stops = stopId)
    cleanD.append(doc)
    
"Done"

100%|██████████| 20000/20000 [00:14<00:00, 1408.08it/s]


'Done'

In [12]:
print(D[100], end='\n\n')
print(cleanD[100], end='\n\n')

RT @TajYasinMZ: Korupsi selalu menjadi permasalahan yang utama di Negara Indonesia. Dengan adanya korupsi, tidak ada negara yang bisa maju.…

korupsi permasalahan utama negara indonesia korupsi negara maju



In [13]:
# Save ke txt, lalu nanti akan di upload ke Voyant Tools

filename = 'projek_2021.txt'
with open(filename, 'w') as f:
    for T in cleanD:
        f.write(T+'\n')
        
"Tweets Saved!"

'Tweets Saved!'

# **Analisis Hashtags**

In [7]:
# Data hashtag nya saja yang ada di tweets

hashtag = []
for i in range(len(T2)):
  if T2[i]['entities']['hashtags'] != []:
    for j in range(len(T2[i]['entities']['hashtags'])):
      hashtag.append(T2[i]['entities']['hashtags'][j]['text'])

hashtag

['hukum_mati',
 'Hukum_mati',
 'AniesGagalTotal',
 'PuanNonKemampuan',
 'HidupTaliban',
 'BUMMTerusKerja',
 'AniesGagalTotal',
 'pln',
 'kpk',
 'lhkpn',
 'kekayaan',
 'petrominer',
 'pln',
 'kpk',
 'lhkpn',
 'kekayaan',
 'petrominer',
 'AniesGagalTotal',
 'HidupTaliban',
 'HidupTaliban',
 'KPKUpdates',
 'SaveKPK',
 'KPK',
 'HidupTaliban',
 'PuanNonKemampuan',
 'KPKUpdates',
 'SaveKPK',
 'KPK',
 'KPKUpdates',
 'SaveKPK',
 'KPK',
 'KPKUpdates',
 'SaveKPK',
 'KPK',
 'Pertamina',
 'HidupTaliban',
 'KPK_RI',
 'HidupTaliban',
 'HidupTaliban',
 'kultwit',
 'HidupTaliban',
 'HidupTaliban',
 'nyimak',
 'HidupTaliban',
 'HidupTaliban',
 'BubarkanKPK',
 'HidupTaliban',
 'mountainDay',
 'kultwit',
 'bintan',
 'korupsi',
 'kpk',
 'bintan',
 'korupsi',
 'kpk',
 'AniesGagalTotal',
 'AniesGagalTotal',
 'TOPNEWS',
 'AntiKorupsi',
 'KPKUpdates',
 'AniesGagalTotal',
 'KPKUpdates',
 'KPKUpdates',
 'KPKUpdates',
 'KPKUpdates',
 'KPKUpdates',
 'PartaiPerindo',
 'KPKRI',
 'KPK',
 'FirliBahuri',
 'HaryTanoeso

In [8]:
# Save ke txt, lalu nanti akan di upload ke Voyant Tools

filename = 'hashtag.txt'
with open(filename, 'w') as f:
    for T in hashtag:
        f.write(T+'\n')
        
"File Saved!"

'File Saved!'

In [32]:
# Melihat tweet dengan keyword tertentu pada data tweet korupsi

Daftar = []
Topik = "#PentingnyaOtsusPapua"
for i in range(len(D)) : 
  if Topik in D[i] : 
    Daftar.append(D[i])

print("Jumlah Kalimat yang mengandung {} adalah {} Data".format(Topik,len(Daftar)))
for Tweet in Daftar:
  print(Tweet)

Jumlah Kalimat yang mengandung #PentingnyaOtsusPapua adalah 54 Data
KOMPAK mendesak Komisi Pemberantasan Korupsi (KPK), untuk menangkap oknum-oknum pejabat negara, yang diduga melakukan tindak pidana korupsi di Provinsi Papua. #PentingnyaOtsusPapua #PapuaIndonesia https://t.co/17uJJIsfku
KOMPAK mendesak Komisi Pemberantasan Korupsi (KPK), untuk menangkap oknum-oknum pejabat negara, yang diduga melakukan tindak pidana korupsi di Provinsi Papua. #PentingnyaOtsusPapua #PapuaIndonesia
#fggsa https://t.co/xMADiRFG40
ForumKOMPAK mendesak Komisi Pemberantasan Korupsi (KPK), untuk menangkap oknum-oknum pejabat negara, yang diduga melakukan tindak pidana korupsi di Provinsi Papua. #PentingnyaOtsusPapua #PapuaIndonesia https://t.co/AlWKKxOHAB
KOMPAK menilai, pengawasan pemerintah belum efektif, sistem birokrasi terus membuka peluang bagi korupsi. #PentingnyaOtsusPapua #PapuaIndonesia #nwofycn https://t.co/arUCIq8JXy
KOMPAK menilai, pengawasan pemerintah belum efektif, sistem birokrasi terus memb